In [ ]:
import pandas as pd

# 1. Chargement des données
df = pd.read_csv("movies.csv")

# 2. Sélection des variables utiles (avant sortie du film)
cols = [
    "budget",
    "runtime",
    "genres",
    "popularity",
    "vote_average",
    "vote_count",
    "original_language",
    "revenue"
]
df = df[cols]

# 3. Nettoyage des données
df.dropna(inplace=True)
df = df[df["budget"] > 0]
df = df[df["runtime"] > 0]

# 4. Création de la variable cible
df["success"] = (df["revenue"] > df["budget"]).astype(int)

# 5. Suppression de la variable revenue (data leakage)
df.drop(columns=["revenue"], inplace=True)

# 6. Vérification finale
print(df.info())
print(df.shape)
print(df["success"].value_counts())

# 7. Export du dataset final
df.to_csv("dataset_final.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 3762 entries, 0 to 4799
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             3762 non-null   int64  
 1   runtime            3762 non-null   float64
 2   genres             3762 non-null   object 
 3   popularity         3762 non-null   float64
 4   vote_average       3762 non-null   float64
 5   vote_count         3762 non-null   int64  
 6   original_language  3762 non-null   object 
 7   success            3762 non-null   int64  
dtypes: float64(3), int64(3), object(2)
memory usage: 264.5+ KB
None
(3762, 8)
success
1    2438
0    1324
Name: count, dtype: int64
